## Introduction

This notebook will cover some technical aspects of Python with regards to common calculations needed in Program Evaluation.

Specifically:

- Label Encoding (making dummies)


- Conditional Mean Calculations
    - Especially in cases where you have missing values


- Regressions using StatsModels
    - Especially in cases where you have missing values


- Numerical Types (float64)
    - Essential for using StatsModels


- T-test for means of two independent samples (SciPy)

The hope is that this notebook will clear up syntax and nuances of using Python over STATA. Python gives you an unprecedented amount of fidelity with regards to calculations - the downside of this freedom is that you must be very careful and explicit about exactly what you are trying to calculate/achieve.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import LabelEncoder
plt.rcParams["figure.figsize"] = (15, 25)
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["axes.labelsize"] = 20
plt.rcParams['legend.fontsize'] = 20
%matplotlib inline

In [2]:
data = pd.read_csv("3-Year_Recidivism_for_Offenders_Released_from_Prison.csv")
data.head()

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Sex,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Release Type,Main Supervising District,Recidivism - Return to Prison,Recidivism Type,Days to Recidivism,New Conviction Offense Classification,New Conviction Offense Type,New Conviction Offense Sub Type,Part of Target Population
0,2010,2013,White - Non-Hispanic,M,Under 25,D Felony,Violent,Assault,Parole,4JD,Yes,Tech,16.0,NaN,NaN,NaN,Yes
1,2010,2013,White - Non-Hispanic,M,55 and Older,D Felony,Public Order,OWI,Parole,7JD,Yes,Tech,19.0,NaN,NaN,NaN,Yes
2,2010,2013,White - Non-Hispanic,M,25-34,D Felony,Property,Burglary,Parole,5JD,Yes,Tech,22.0,NaN,NaN,NaN,Yes
3,2010,2013,White - Non-Hispanic,M,55 and Older,C Felony,Drug,Trafficking,Parole,8JD,Yes,Tech,25.0,NaN,NaN,NaN,Yes
4,2010,2013,Black - Non-Hispanic,M,25-34,D Felony,Drug,Trafficking,Parole,3JD,Yes,Tech,26.0,NaN,NaN,NaN,Yes


In the snapshot of data below, we have two columns that need to be turned into dummy variables.

There are various ways to achieve this using Pandas ([`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)), but you can also use functions from sci-kit learn.

Below, since we only have the binary case, im going to use `LabelEncoder` from sci-ket learn. Of course, you can also use one-hot encoding.

For a discussion over the differences between Label Encoding and One-HotEncoding see [this](https://stackoverflow.com/questions/38413579/what-is-the-difference-between-sklearn-labelencoder-and-pd-get-dummies) and [this.](https://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor)

Label encoder documentation: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

One Hot encoding documentation: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder

In [3]:
data[["Recidivism - Return to Prison", "Part of Target Population"]].tail()

,Recidivism - Return to Prison,Part of Target Population
21641,No,Yes
21642,Yes,Yes
21643,No,No
21644,No,No
21645,Yes,Yes


Below, you can see that we have successfully encoded "Yes" to 1 and "No" to 0.

In [4]:
encoder = LabelEncoder()
data['Y'] = encoder.fit_transform(data['Recidivism - Return to Prison'])
data['D'] = encoder.fit_transform(data['Part of Target Population'])
data[['Recidivism - Return to Prison', 'Y', 'Part of Target Population', 'D']].tail()

,Recidivism - Return to Prison,Y,Part of Target Population,D
21641,No,0,Yes,1
21642,Yes,1,Yes,1
21643,No,0,No,0
21644,No,0,No,0
21645,Yes,1,Yes,1


It is also possible to apply a more sophisticated encoding function to a partiuclar column by utilising the `.apply` method in conjunction with lambda functions.

In [5]:
def fancy_encoding(val):
    if val == "No":
        return 0
    else:
        return 1

data['fancy_Y'] = data['Recidivism - Return to Prison'].apply(lambda x: fancy_encoding(x))

In [6]:
data[['Recidivism - Return to Prison', 'Y', 'fancy_Y']].tail()

,Recidivism - Return to Prison,Y,fancy_Y
21641,No,0,0
21642,Yes,1,1
21643,No,0,0
21644,No,0,0
21645,Yes,1,1


Now, suppose we wish to calcualte a condition expectation - how might we use the power of pandas to help with this?

By using [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

In [7]:
subset = data[['Y','D']]
subset[subset['D'] == 1].groupby('Y').count()

,D
Y,
0,7131
1,4337


$\frac{7137}{11474} = 0.6220149904131079$


$\frac{4337}{11474} = 0.37798500958689213$


$$E[Recidivist | Participant] = E[Y | D = 1] = 0 \cdot 0.6220149904131079 + 1 \cdot 0.37798500958689213 = 0.37798500958689213$$

Now, suppose we need to calculate the mean of a certain feature.

By default, you may reach for `np.mean` - however, this may not have the behaviour you expect if that feature has missing values.

In [8]:
new_data = pd.read_stata("MTO_data.dta") #reading in a stata file
col_list = ["male", "head_earnings", "bmi_40","no_family_in_nbhd", "anxiety"]
new_data[col_list].head()

,male,head_earnings,bmi_40,no_family_in_nbhd,anxiety
0,1.0,-38480.976562,0=Not Obese3,0.0,1=Generalized Anxiety Disorder (past yr)
1,1.0,21206.250000,1=Obese3,1.0,1=Generalized Anxiety Disorder (past yr)
2,0.0,13893.000977,1=Obese3,1.0,0=No Generalized Anxiety Disorder (past yr)
3,0.0,15571.154297,0=Not Obese3,1.0,1=Generalized Anxiety Disorder (past yr)
4,0.0,11620.408203,0=Not Obese3,0.0,0=No Generalized Anxiety Disorder (past yr)


In [9]:
any(new_data['head_earnings'].isnull()) # we have missing data in this column!

True

In [10]:
mean_1 = np.mean(new_data['head_earnings'])
mean_1

12499.498046875

In [11]:
mean_2 = np.nanmean(new_data['head_earnings'])
mean_2

12499.478

In [12]:
mean_1 == mean_2

False

In [13]:
new_data["head_earnings"].notnull().sum() # The number of missing values in the head earnings column!

2986

Now, this might seem like a minute difference, however, precision in these types of calculations are paramount when running calculations related to p-values.

The difference comes from the fact that [`np.mean`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html) with case `NaN` values to 0 whereas [`np.nanmean`](https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanmean.html) will completly ignore `NaN` values from the calculation.

There is also a [`np.nanstd`](https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanstd.html) function for calculating the standard deviation.

When doing regressions, I highly recommend using the [StatsModels library](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) as this will spit out output that is almost identical to R/STATA. You can see example of this in my previous [notebook](https://github.com/igabr/Blog_Post_Notebooks/blob/master/Program_Evaluation_Introduction_Notebook.ipynb).

An important thing to note is then when running regressions with missing data, STATA will automatically drop rows that contain `NaNs` - this is NOT the case with statsmodels.

In fact, should you data contain `NaNs`, you will just see `NaNs` populating you output table.

Rather, you must specify what to do with missing data.

`model = sm.OLS(outcome, X, missing='drop')`

Let now talk datatypes:

In [14]:
new_data.dtypes.unique()

array([dtype('float32'), category], dtype=object)

From the above, we can see that our dataframe contains `float32` and `category` datatypes.

Off the bat, this might seem great, but it is important to remember that when you start encoding various variables or doing transformations that the data type of your features may change.

Before feeding in features to a regression, I like to ensure that all of my numeric columns are encoded to `float64` as this gives more numerical precision.

You can change the type of a column as follows:

In [15]:
new_data['head_earnings'].dtype

dtype('float32')

In [16]:
new_data['head_earnings'] = new_data['head_earnings'].astype('float64')

In [17]:
new_data['head_earnings'].dtype # ta-da!

dtype('float64')

You can also use method chaining after to an `.apply` method to immeadiately cast the result of your transformation to a `float64`.

e.g. `data[col_name] = data[col_to_be_transformed].apply(lambda x: fancy(x)).astype("float64")` 

In the context of Program Evaluation and Causal Inference, we are more interested in looking at the predicted values of our regression on the data we train the regression on. This is opposed to Machine Learning in which we simply train a model and our interest lies in the predicted values of **test data** we feed it.

That is to say, we are concerned here with **fitted values** of our model. You can obtain the fitted values of your regression by using a method:

```{python}
results = model.fit()
data['fitted_values'] = results.fittedvalues
```

**fitted values** are simply the values that lie on the regression line. Accessing these values will be important when wishing to verify conditional mean calculations by way of bivariate regressions.

Lastly, if is often the case that we need to run t-tests in Program Evaluation. While a majority of this can be handled via statsmodels output. Sometimes, we wish to run a t-test on 2 sample populations. In these calculations, it is important that we are careful with the calculation of $N$, standard deviations and the mean.

If we manually and accurately calculate these with `np.nanmean`, `np.nanstd` and `df[c].notnull().sum()`, then we can make use of [`ttest_ind_from_stats`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind_from_stats.html)

In the case that we have cleaner data where we can be sure there will be no tainting of our calculations, we can make use of [`ttest_ind`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html). Furthermore, this function does have an optional argument that can specify how to deal with `NaN` values.

This function is significantly easier to use as we only need to provide the 2 sources of data as opposed to the 6 input parameters required from `ttest_ind_from_stats`.

## Conclusion

That's all for this notebook. I hope you find it to be a timesaving resource with regards to syntax and 'gotcha's' due to NaNs when running your own analysis.

As always, feel free to reach out at: igabr@uchicago.edu or [@Gabr\_Ibrahim](https://twitter.com/Gabr_Ibrahim)